# CKD Plan Designer Tutorial

## Install CKDPlanDesigner Package

In [63]:
! pip install -e project_code/

Obtaining file:///Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code
  Preparing metadata (setup.py) ... done
  Attempting uninstall: CKDPlanDesigner
    Found existing installation: CKDPlanDesigner 0.0.0
    Uninstalling CKDPlanDesigner-0.0.0:
      Successfully uninstalled CKDPlanDesigner-0.0.0
  Running setup.py develop for CKDPlanDesigner


## Load Data
- This tutorial uses the freely available synthea example dataset
    - Download it here: https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
    - unzip it to SYNTHEA_PATH
 

In [64]:
import pandas as pd
import os

from CKDPlanDesigner.utils import data_preprocessing as pre

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
SYNTHEA_PATH = '/Users/nicholasdeveau/projects/bmi210/bmi210Project_data/synthea_data/'

#### Here we build our patient dataframe
- This contains a vecor for each patient containing the required information for plan designing 

In [69]:
patients_df = pre.preprocess_synthea_data(SYNTHEA_PATH)

In [71]:
patients_df.head()

,patient_id,t2d,depression,bmi,hypertension,smoking,dob,race,ethnicity,gender,zip,eGFR
0,c8403116-d069-4130-aa02-39954fc971b5,True,False,False,False,False,1967-03-26,white,nonhispanic,F,NaN,66.9
1,73f124a4-e69e-4b8c-a447-e72d23dc54be,True,False,False,True,True,1967-06-14,white,nonhispanic,F,2170.0,70.5
2,31459cc4-45ed-4dee-b160-9f1a148a7724,False,False,True,False,False,1968-08-15,white,nonhispanic,M,NaN,75.4
3,1d7fe784-8667-47e8-b733-8e6ec0fbb5f9,False,False,True,False,False,1940-04-05,white,nonhispanic,F,NaN,83.0
4,cdef53d5-0537-4073-8874-79fa5e0346e6,True,False,True,True,True,1940-04-05,white,nonhispanic,F,NaN,145.2


## Working With a Patient

In [72]:
from CKDPlanDesigner.models import patients
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
patient1_config = patients_df.to_dict(orient='index')[0]

In [77]:
patient1_config

{'patient_id': 'c8403116-d069-4130-aa02-39954fc971b5',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1967-03-26',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 66.9}

In [76]:
# patient_config = {'patient_id': 123,2
#                   'eGFR': 70,
#                   'depression': True
#                  }


# select the first patient and convert feature vector to a dictionary
# we pass this dictionary to the patient class on instantiation
patient1_config = patients_df.to_dict(orient='index')[0]
p1 = patients.Patient(**patient1_config)

patient2_config = patients_df.to_dict(orient='index')[40]
p2 = patients.Patient(**patient2_config)

#### Look at patient 1's charecteristics

In [54]:
p1.patient_config

{'patient_id': 'c8403116-d069-4130-aa02-39954fc971b5',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1967-03-26',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 66.9}

In [55]:
# we automatically assign a CKD stage when a patient is created
p1.stage

### Generate a care plan

In [83]:
p1.generate_careplan()
print(p1.careplan.plan_name)

Plan: Delay


In [86]:
p1.patient_config

{'patient_id': 'c8403116-d069-4130-aa02-39954fc971b5',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1967-03-26',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 66.9}

In [84]:
p1.careplan

In [85]:
p1.careplan.list_components()

Plan Components


[('T2D Management', 'Cost Reduction: 0.65')]

### Try it out for a different patient
- This patient has more co-morbidities. Check them out!
- Because of this, we should expect a more comprehensive care plan even though they're still just stage 2

In [58]:
p2.patient_config

{'patient_id': nan,
 't2d': True,
 'depression': True,
 'bmi': True,
 'hypertension': True,
 'smoking': True,
 'dob': nan,
 'race': nan,
 'ethnicity': nan,
 'gender': nan,
 'zip': nan,
 'eGFR': nan}

In [59]:
p2.stage

In [87]:
p2.generate_careplan()
p2.careplan.list_components(show_savings=False)

Plan Components


['Hypertension Management',
 'T2D Management',
 'Additional Time - Health Equity',
 'Depression Treatment',
 'Dietary Program']

In [88]:
p2.patient_config

{'patient_id': nan,
 't2d': True,
 'depression': True,
 'bmi': True,
 'hypertension': True,
 'smoking': True,
 'dob': nan,
 'race': nan,
 'ethnicity': nan,
 'gender': nan,
 'zip': nan,
 'eGFR': nan}

In [89]:
p2.stage

In [91]:
p2.careplan.list_components(show_savings=False)

Plan Components


['Hypertension Management',
 'T2D Management',
 'Additional Time - Health Equity',
 'Depression Treatment',
 'Dietary Program']

In [ ]:
"""
1) profile patient (patient_config)
2) classified into disease (patient.stage)
3) unoptimized charecteristics (trey creating this in the ontology)
    3a) unoptimmized = some min number of "bad" criteria (from patient_config)
"""